In [1]:
!pip install progress
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for progress: filename=progress-1.6-py3-none-any.whl size=9630 sha256=177b7de78ed19e37428762186ac8bac01c7844d81bf73fb9be568f9c9cb9a9a8
  Stored in directory: /root/.cache/pip/wheels/4c/9b/0a/a78ff56725af3ef70792f9ed0f8dbbc4c0315edc62cbc4a6b8
Successfully built progress
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.2 MB/s eta 0:00:00


In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')

print("curren directory:", os.getcwd())
BASE_DIR = '/content/drive/MyDrive/AdversarialAI/MixMatch/'
FIXMATCH_DIR = BASE_DIR + ""
os.chdir(FIXMATCH_DIR)
print("changed to:", os.getcwd())
import sys
BASE_DIR = '/content/drive/MyDrive/AdversarialAI/'
sys.path.insert(1, BASE_DIR + 'Poisoner')
sys.path.insert(1, BASE_DIR + 'MixmMatch')

Mounted at /content/drive
curren directory: /content
changed to: /content/drive/MyDrive/MixMatch


In [3]:
from scipy import stats
from torch.utils.data import DataLoader, ConcatDataset, Dataset
from torchvision import transforms, datasets
import numpy as np
import imageio
from PIL import Image

from CustomTransforms import get_transforms
from get_datasets import get_subset_cifar10, CIFAR_POISON,get_subset_mnist,get_subset_fashion_mnist
import os
import shutil
import pdb
from Poisoner import Poisoner

['/content', '/content/drive/MyDrive/AdversarialAI/MixmMatch', '/content/drive/MyDrive/AdversarialAI/Poisoner', '/env/python', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.9/dist-packages/IPython/extensions', '/root/.ipython', './']


In [4]:
import torch
from torch.utils.data import DataLoader, SequentialSampler
import torch.nn.functional as F
from utils import AverageMeter, accuracy
from tqdm import tqdm
import models.wideresnet as models
import time
import logging

import sys
sys.path.insert(1, BASE_DIR + 'Poisoner')
from Poisoner import Poisoner
from get_datasets import get_subset_cifar10
from CustomTransforms import get_fixmatch_transforms, get_mixmatch_transforms

# logger
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO)

# prepare cuda
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"
device = "cpu" # always use cpu
print(f"Using {device}")



Using cpu


In [5]:
def test_on_testset(test_loader, model, device):
    logger = logging.getLogger(__name__)
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    end = time.time()

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            data_time.update(time.time() - end)
            model.eval()

            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, targets)

            prec1, prec5 = accuracy(outputs, targets, topk=(1, 5))
            losses.update(loss.item(), inputs.shape[0])
            top1.update(prec1.item(), inputs.shape[0])
            top5.update(prec5.item(), inputs.shape[0])
            batch_time.update(time.time() - end)
            end = time.time()

    logger.info("top-1 acc: {:.2f}".format(top1.avg))
    logger.info("top-5 acc: {:.2f}".format(top5.avg))
    return losses.avg, top1.avg




In [6]:
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
cifar10_label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
fashion_mnist_label_names = ["Tshirt", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
mnist_label_names = ["Zero", "One", "Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine"]

dataset = "mnist"

if dataset=="cifar10":
  label_names = cifar10_label_names
elif dataset =="mnist":
  label_names = mnist_label_names
elif dataset =="fashion_mnist":
  label_names = fashion_mnist_label_names

label_pair = [2,6]
DIR = ""
N = 150
[label_names[label_pair[0]],label_names[label_pair[1]]]

['Two', 'Six']

In [7]:
# load model
model = models.WideResNet(num_classes=10)
model.to(device)
CHECKPOINT_PATH = f"result/{dataset}_{N}poison_{label_pair[0]}_{label_pair[1]}_mixmatch_model_best.pth.tar"

checkpoint = torch.load(CHECKPOINT_PATH,map_location=torch.device(device))
model.load_state_dict(checkpoint['state_dict'])
model.eval()
print("Loaded model at", CHECKPOINT_PATH)

# load transforms
transform_labeled, transform_unlabeled, transform_test = get_mixmatch_transforms(dataset=dataset)

Loaded model at result/mnist_150poison_2_6_mixmatch_model_best.pth.tar


In [8]:
# load test data

if dataset=="cifar10":
  labeled_dataset, unlabeled_dataset, test_dataset = get_subset_cifar10()
elif dataset =="mnist":
  labeled_dataset, unlabeled_dataset, test_dataset = get_subset_mnist()
elif dataset =="fashion_mnist":
  labeled_dataset, unlabeled_dataset, test_dataset = get_subset_fashion_mnist()


test_dataset.transform = transform_test
test_dataset.asPIL = False

test_loader = DataLoader(test_dataset,
                        sampler=SequentialSampler(test_dataset),
                        batch_size=8,
                        num_workers=1)

# test
test_on_testset(test_loader, model, device)

Getting MNIST | Subset size, labeled size, test size = [20000, 200, 4000]
labeled, unlabeled, test sizes: [200, 19800, 3600]


(0.08081001698763834, 98.05555555555556)

In [9]:
from PIL import Image
import numpy as np



poison_output = f"./data/latest_poison_{dataset}_{label_pair[0]}_{label_pair[1]}_{N}"

CHOSEN_POISON_PATH = poison_output+f'/mali_{label_names[label_pair[0]]}.jpg'



poisoner = Poisoner()
poison_dataset= poisoner._read_stored_samples(input_folder=poison_output,target_sample=label_pair[1])
transform_labeled, transform_unlabeled, transform_test = get_transforms(method="mixmatch",dataset=dataset)
poison_dataset.transform = transform_test
n = poison_dataset.__len__()
pred_acc =[]
for i in range(n):
  im, label = poison_dataset.__getitem__(i)
  inputs = im.unsqueeze(0)
  
  outputs = model(inputs)
  outputs = torch.softmax(outputs, dim=1)
  # print(outputs)
  label_idx = torch.argmax(outputs).cpu().numpy()
  label = label_names[label_idx]
  pred_acc.append(label_idx==label_pair[1])
  # print("prediction:", label)

# march5night --> Fail 50k.10
print(np.mean(pred_acc))

0.10869565217391304


In [10]:
injected_img = imageio.imread(CHOSEN_POISON_PATH)
if injected_img.ndim ==2:
    injected_img = np.expand_dims(injected_img, axis=0)
X_adv = np.array([injected_img])
y_adv = label_pair[1]
poison_dataset = CIFAR_POISON(data=X_adv, label=y_adv, transform=None)
poison_dataset.transform = transform_test

im, label = poison_dataset.__getitem__(0)
inputs = im.unsqueeze(0)
outputs = model(inputs)
outputs = torch.softmax(outputs, dim=1)
# print(outputs)
label_idx = torch.argmax(outputs).cpu().numpy()
label = label_names[label_idx]
label_idx==label_pair[1]

<ipython-input-10-0a29b5b3d74b>:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  injected_img = imageio.imread(CHOSEN_POISON_PATH)


False

Evaluating the attack success on the test set

In [11]:
# for i in range(n):
#   im, label = test_dataset.__getitem__(i)
#   if label != label_names.index(label_pair[0]):
#     continue
#   inputs = im.unsqueeze(0)
  
#   outputs = model(inputs)
#   outputs = torch.softmax(outputs, dim=1)
#   # print(outputs)
#   label_idx = torch.argmax(outputs).cpu().numpy()
#   label = label_names[label_idx]
#   pred_acc.append(label==label_pair[1])
#   print("prediction: ", "Succeeded" if label==label_pair[1] else "Failed" )